## Imports

In [1]:
import tensorflow as tf

from tensorflow.keras.layers import LSTM

# from tensorflow.keras.models import Sequential

from tensorflow.keras.losses import MeanSquaredError
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from keras.preprocessing.sequence import TimeseriesGenerator

#import np.array as array



## Load Datasets
##### As Pandas DataFrame objects

In [4]:
training_path = "D:\OneDrive\OneDrive - University of Massachusetts Dartmouth\ECE457_Senior_Design_ECE5\CSV_Files\PortTraining_1404_002.csv"
#training_path = "D:\OneDrive\OneDrive - University of Massachusetts Dartmouth\ECE457_Senior_Design_ECE5\CSV_Files\\train1_port_new.csv"
test_path = "D:\OneDrive\OneDrive - University of Massachusetts Dartmouth\ECE457_Senior_Design_ECE5\CSV_Files\StbdTesting_1404_002.csv"
#test_path = "D:\OneDrive\OneDrive - University of Massachusetts Dartmouth\ECE457_Senior_Design_ECE5\CSV_Files\\test1_stbd_new.csv"

In [5]:
# Load Dataset
#import csv as a pandas dataframe
#training_path = "D:\OneDrive\OneDrive - University of Massachusetts Dartmouth\ECE457_Senior_Design_ECE5\CSV_Files\Training_Port1.csv"

# test_path = "D:\OneDrive\OneDrive - University of Massachusetts Dartmouth\ECE457_Senior_Design_ECE5\CSV_Files\Test_Stbd1.csv"
df_train=pd.read_csv(training_path)
df_test=pd.read_csv(test_path)

In [8]:
pd.get_option("display.max_columns")

In [7]:
pd.set_option("display.max_columns", None)

In [9]:
df_train.shape


(5504457, 32)

In [11]:
df_train.head(5)

,PingNum,SampleNum,PortStbd,SampleTimeDelay,I1,Q1,I2,Q2,I3,Q3,I4,Q4,I5,Q5,I6,Q6,I7,Q7,I8,Q8,I9,Q9,I10,Q10,Roll,C,DOA,TWTT,Amplitude,AngleUncertainty,SampleRate,Range
0,87443,5,0,0.000061,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.488892,1492.04,-55.809999,0.000061,0,0,65108.40625,0.045832
1,87443,6,0,0.000077,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.488892,1492.04,-55.809999,0.000077,0,0,65108.40625,0.057291
2,87443,7,0,0.000092,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.488892,1492.04,-55.809999,0.000092,0,0,65108.40625,0.068749
3,87443,8,0,0.000108,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.488892,1492.04,-55.809999,0.000108,0,0,65108.40625,0.080207
4,87443,9,0,0.000123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.488892,1492.04,-55.809999,0.000123,0,0,65108.40625,0.091665


In [10]:
df_train.tail(5)


,PingNum,SampleNum,PortStbd,SampleTimeDelay,I1,Q1,I2,Q2,I3,Q3,I4,Q4,I5,Q5,I6,Q6,I7,Q7,I8,Q8,I9,Q9,I10,Q10,Roll,C,DOA,TWTT,Amplitude,AngleUncertainty,SampleRate,Range
5504452,88723,4297,0,0.065982,-28.2500,24.0000,-4.6875,41.8750,23.7500,30.8125,42.0625,-26.6250,-6.8125,-60.3750,-52.8125,-18.4375,-22.4375,27.6875,33.4375,12.6250,18.6250,-25.0625,-5.6875,-2.3125,1.016235,1491.949,-82.134998,0.065982,76,206,65108.40625,49.221089
5504453,88723,4298,0,0.065998,-70.8750,26.7500,7.2500,78.1250,66.7500,30.7500,58.2500,-53.7500,-36.6250,-73.9375,-88.6875,-1.8750,-21.6250,64.3125,67.5000,31.6250,46.7500,-57.0625,-43.3125,-50.6875,1.016235,1491.949,-80.939998,0.065998,84,97,65108.40625,49.232546
5504454,88723,4299,0,0.066013,-42.5625,15.3750,14.5625,48.1875,42.7500,7.6250,14.0625,-45.5625,-50.3125,-30.6250,-56.5000,23.4375,1.0625,53.5625,44.3125,20.3750,37.2500,-45.0000,-46.6875,-43.5000,1.016235,1491.949,-80.624998,0.066013,96,32,65108.40625,49.244004
5504455,88723,4300,0,0.066028,35.9375,-15.7500,13.1875,-39.5000,-40.5625,-27.1250,-38.9375,24.8125,22.6250,30.6875,41.5625,-21.3750,-4.6250,-27.9375,-31.6250,7.9375,-0.6875,22.1250,1.4375,16.5625,1.016235,1491.949,-80.924998,0.066028,90,48,65108.40625,49.255461
5504456,88723,4301,0,0.066044,68.6250,-22.3125,20.3125,-71.5000,-57.5000,-54.6875,-57.0625,52.8750,61.3750,61.6875,101.0000,-44.3125,0.2500,-87.8125,-73.1875,-11.5625,-47.3750,67.8125,39.2500,62.3125,1.016235,1491.949,-79.104998,0.066044,84,113,65108.40625,49.266919


In [21]:
MaxPingNumTrain = df_train["PingNum"].max()
MaxPingNumTest = df_test["PingNum"].max()
print("Max Ping Num : Train = ", MaxPingNumTrain, ", Test = ", MaxPingNumTest)

Max Ping Num : Train =  88723 , Test =  88723


In [22]:
MinPingNumTrain = df_train["PingNum"].min()
MinPingNumTest = df_test["PingNum"].min()
print("Min Ping Num : Train = ", MinPingNumTrain, ", Test = ", MinPingNumTest)

Min Ping Num : Train =  87443 , Test =  87443


In [24]:
df_train.shape

(5504457, 32)

In [25]:
df_test.head()

,PingNum,SampleNum,PortStbd,SampleTimeDelay,I1,Q1,I2,Q2,I3,Q3,I4,Q4,I5,Q5,I6,Q6,I7,Q7,I8,Q8,I9,Q9,I10,Q10,Roll,C,DOA,TWTT,Amplitude,AngleUncertainty,SampleRate,Range
0,87443,5,1,0.000061,0.0000,0.0000,0.000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.488892,1492.04,-55.809999,0.000261,0,0,65108.40625,0.194788
1,87443,19,1,0.000276,0.0000,0.0000,0.000,0.0625,0.0000,0.125,0.0000,0.1250,0.0000,0.0625,0.0000,0.0000,0.0000,0.0000,-0.0625,0.0000,0.0000,0.0000,0.0000,0.0000,0.488892,1492.04,-59.234999,0.000276,0,126,65108.40625,0.206246
2,87443,20,1,0.000292,0.0625,0.0000,0.125,0.0000,0.1875,0.000,0.1875,-0.0625,0.1250,0.0000,0.0000,0.0000,0.0000,0.0625,0.0000,0.0625,0.0000,0.0000,0.0000,0.0000,0.488892,1492.04,-59.234999,0.000292,0,83,65108.40625,0.217704
3,87443,21,1,0.000307,0.0000,-0.0625,0.000,-0.1875,-0.0625,-0.250,-0.1875,-0.2500,-0.0625,-0.1250,0.0000,-0.0625,0.0625,-0.0625,0.1250,0.0000,0.0625,0.0000,0.0000,-0.0625,0.488892,1492.04,-59.234999,0.000307,0,69,65108.40625,0.229162
4,87443,22,1,0.000323,-0.0625,0.0000,-0.250,0.0625,-0.3125,0.125,-0.2500,0.3125,-0.1875,0.1250,-0.0625,0.0000,-0.0625,-0.0625,0.0000,-0.1875,0.0000,-0.0625,-0.0625,0.0000,0.488892,1492.04,-58.524999,0.000323,0,120,65108.40625,0.240621


In [ ]:
# Model Configuration
#addition_metrics = ['accuracy']
#batch_size = 128
#embedding_output_dims = 15
#loss_function = MeanSquaredError()
#max_sequence_length = 300
#num_distinct_words = 5000
#number_of_epochs = 5
#optimizer = Adam()
#validation_split = 0.20
#verbosity_mode = 1

# Function Declarations

In [33]:
# Function to get a given ping #'s data and put it into corresponding locations.
def GetPingNumber(pingnum, source):
    if (source == 'train'):
        df = df_train[df_train['PingNum']==pingnum]
    if (source == 'test'):
        df = df_test[df_test['PingNum']==pingnum]
        
    return df


In [27]:
# Function to get the important columns out of a dataframe object..
def GetColumns(in_df_obj, ColumnTypes):
    if (ColumnTypes == 'InputData'):
        df1 = in_df_obj.iloc[:,3:24]    # SampleTimeDelay through (including) Roll.
        df2 = in_df_obj.iloc[:,28]      # 28 is Amplitude values
        df = pd.concat([df1, df2], axis=1)
        #df1 = in_df_obj[in_df_obj.iloc[:,3:24]]     # 3:24 contain the SampleTimeDelay and IQ vals.
        #df2 = in_df_obj[in_df_obj.iloc[:,29]]       # 29 is the Amplitude value
        #df = df1.append(df2)
        
    if (ColumnTypes == 'TrueData'):
        df = in_df_obj.iloc[:,26]                   # 26 is the DOA values
        #df = in_df_obj[in_df_obj.iloc[27]]
        #colRange1 = 

    if (ColumnTypes == 'FirstFew'): # this returns only columns 'Ping#', 'Sample#', 'Port/Stbd', and 'SampleTimeDelay'. (To Scale between 0 and 1)
        df = in_df_obj.iloc[:,0:4]

    if (ColumnTypes == 'IQandROLL'): #This only returns the I & Q Data columns and the Roll data column (To Scale between -1 and 1)
        df = in_df_obj.iloc[:, 4:25] 

    if (ColumnTypes == 'C'):        # this returns only the 'C' column (To Scale between 0 and 1)
        df = in_df_obj.iloc[:, 25]

    if (ColumnTypes == 'DOA'):      # this returns only the 'DOA' column (To Scale between -1 and 1)
        df = in_df_obj.iloc[:, 26]

    if (ColumnTypes == 'LastFew'):  # this returns only the 'TWTT', 'Amplitude', 'AngleUncertainty', 'SampleRate', and 'Range' columns. (To Scale between 0 and 1)
        df = in_df_obj.iloc[:, 27:32]

    return df


In [28]:
def select_columns(data_frame, column_names):
    new_frame = data_frame.loc[:, column_names]
    return new_frame

In [29]:
scaler = MinMaxScaler()
#scaler_neg1_1 = MinMaxScaler((-1,1))
scaler_neg1_1 = MinMaxScaler((-1,1))

In [30]:
df_train.iat[0,0]


87443

In [36]:
dfPing7 = GetPingNumber(87445, 'train')
dfPing7

,PingNum,SampleNum,PortStbd,SampleTimeDelay,I1,Q1,I2,Q2,I3,Q3,I4,Q4,I5,Q5,I6,Q6,I7,Q7,I8,Q8,I9,Q9,I10,Q10,Roll,C,DOA,TWTT,Amplitude,AngleUncertainty,SampleRate,Range
8594,87445,5,0,0.000061,0.000,0.000,0.000,0.000,0.000,0.000,0.0000,0.0000,0.000,0.000,0.000,0.000,0.0000,0.0000,0.000,0.000,0.000,0.000,0.000,0.000,0.351562,1492.04,-55.809999,0.000061,0,0,65108.40625,0.045832
8595,87445,6,0,0.000077,0.000,0.000,0.000,0.000,0.000,0.000,0.0000,0.0000,0.000,0.000,0.000,0.000,0.0000,0.0000,0.000,0.000,0.000,0.000,0.000,0.000,0.351562,1492.04,-55.809999,0.000077,0,0,65108.40625,0.057291
8596,87445,7,0,0.000092,0.000,0.000,0.000,0.000,0.000,0.000,0.0000,0.0000,0.000,0.000,0.000,0.000,0.0000,0.0000,0.000,0.000,0.000,0.000,0.000,0.000,0.351562,1492.04,-55.809999,0.000092,0,0,65108.40625,0.068749
8597,87445,8,0,0.000108,0.000,0.000,0.000,0.000,0.000,0.000,0.0000,0.0000,0.000,0.000,0.000,0.000,0.0000,0.0000,0.000,0.000,0.000,0.000,0.000,0.000,0.351562,1492.04,-55.809999,0.000108,0,0,65108.40625,0.080207
8598,87445,9,0,0.000123,0.000,0.000,0.000,0.000,0.000,0.000,0.0000,0.0000,0.000,0.000,0.000,0.000,0.0000,0.0000,0.000,0.000,0.000,0.000,0.000,0.000,0.351562,1492.04,-55.809999,0.000123,0,0,65108.40625,0.091665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12886,87445,4297,0,0.065982,27.375,27.750,30.625,-23.000,-20.375,-18.750,-36.9375,32.3750,-2.500,35.625,12.750,14.875,21.7500,-11.4375,0.250,-51.375,-65.125,-20.000,-8.625,63.625,0.335083,1492.04,-80.679998,0.065982,92,73,65108.40625,49.224091
12887,87445,4298,0,0.065998,-102.000,-6.500,-79.250,102.125,48.250,94.125,94.3750,-16.8750,-1.750,-102.500,-91.750,-17.250,-50.4375,88.1875,50.875,40.125,57.375,-47.500,-3.375,-71.250,0.335083,1492.04,-81.344998,0.065998,86,86,65108.40625,49.235549
12888,87445,4299,0,0.066013,-143.375,20.125,-54.375,156.125,123.625,95.875,141.5000,-92.1875,-63.875,-160.000,-170.500,17.625,-45.1250,169.6875,116.500,85.000,121.750,-103.375,-57.125,-142.750,0.335083,1492.04,-80.089998,0.066013,100,44,65108.40625,49.247007
12889,87445,4300,0,0.066028,-123.500,16.625,-18.750,124.500,128.375,71.625,127.2500,-99.8750,-83.250,-133.250,-165.375,27.875,-16.7500,144.8750,120.000,70.625,100.250,-102.625,-76.625,-102.000,0.335083,1492.04,-80.394998,0.066028,108,19,65108.40625,49.258466


In [96]:
df_train_firstFewColumns = df_train

In [97]:
df_train_scaled = pd.DataFrame(scaler.fit_transform(df_train), columns=df_train.columns)

df_train_scaled['Ping#'] = df_train['Ping#']

df_train_scaled['SampleRate'] = df_train['SampleRate']

df_train_scaled['Range'] = df_train['Range']

df_train_scaled['C'] = df_train['C']

df_train_scaled['Port/Stbd'] = df_train['Port/Stbd']


#df_train_scaled['Sample#'] = df_train['Sample#']



df_test_scaled = pd.DataFrame(scaler.fit_transform(df_test), columns=df_test.columns)
df_test_scaled['Ping#'] = df_test['Ping#']
df_test_scaled['SampleRate'] = df_test['SampleRate']
df_test_scaled['Range'] = df_test['Range']
df_test_scaled['C'] = df_test['C']
df_test_scaled['Port/Stbd'] = df_test['Port/Stbd']

#df_train_scaled['Sample#'] = df_train['Sample#']


In [98]:
C_Array = pd.DataFrame(GetColumns(df_train, 'C'))

#C_Array.reshape(-1, 1)

C_Array

,C
0,1492.040
1,1492.040
2,1492.040
3,1492.040
4,1492.040
...,...
5504452,1491.949
5504453,1491.949
5504454,1491.949
5504455,1491.949


In [99]:
FirstFewScaled = pd.DataFrame(scaler.fit_transform(GetColumns(df_train, 'FirstFew')), columns=df_train.columns[0:4])

FirstFewScaled['Ping#']=df_train['Ping#']
FirstFewScaled['Sample#']=df_train['Sample#']
FirstFewScaled['Port/Stbd']=df_train['Port/Stbd']
FirstFewScaled.head()

,Ping#,Sample#,Port/Stbd,SampleTimeDelay
0,7,5,0,0.000000
1,7,6,0,0.000233
2,7,7,0,0.000466
3,7,8,0,0.000698
4,7,9,0,0.000931


In [100]:
IQandROLL = pd.DataFrame(scaler_neg1_1.fit_transform(GetColumns(df_train, 'IQandROLL')), columns=df_train.columns[4:25])
IQandROLL.head()

,I1,Q1,I2,Q2,I3,Q3,I4,Q4,I5,Q5,I6,Q6,I7,Q7,I8,Q8,I9,Q9,I10,Q10,Roll
0,0.126465,0.039453,-0.081946,0.169626,-0.00804,0.072351,-0.072821,-0.014608,-0.02566,-0.024656,-0.049759,-0.005408,-0.052884,-0.020516,-0.067236,-0.119361,0.020121,-0.073706,0.132279,-0.075426,0.147727
1,0.126465,0.039453,-0.081946,0.169626,-0.00804,0.072351,-0.072821,-0.014608,-0.02566,-0.024656,-0.049759,-0.005408,-0.052884,-0.020516,-0.067236,-0.119361,0.020121,-0.073706,0.132279,-0.075426,0.147727
2,0.126465,0.039453,-0.081946,0.169626,-0.00804,0.072351,-0.072821,-0.014608,-0.02566,-0.024656,-0.049759,-0.005408,-0.052884,-0.020516,-0.067236,-0.119361,0.020121,-0.073706,0.132279,-0.075426,0.147727
3,0.126465,0.039453,-0.081946,0.169626,-0.00804,0.072351,-0.072821,-0.014608,-0.02566,-0.024656,-0.049759,-0.005408,-0.052884,-0.020516,-0.067236,-0.119361,0.020121,-0.073706,0.132279,-0.075426,0.147727
4,0.126465,0.039453,-0.081946,0.169626,-0.00804,0.072351,-0.072821,-0.014608,-0.02566,-0.024656,-0.049759,-0.005408,-0.052884,-0.020516,-0.067236,-0.119361,0.020121,-0.073706,0.132279,-0.075426,0.147727


In [101]:
C_COLUMN = pd.DataFrame(scaler.fit_transform(pd.DataFrame(GetColumns(df_train, 'C'), columns=[df_train.columns[25]])), columns=[df_train.columns[25]])
C_COLUMN.head()

#

,C
0,0.90991
1,0.90991
2,0.90991
3,0.90991
4,0.90991


In [102]:
DOA_COLUMN = pd.DataFrame(scaler_neg1_1.fit_transform(pd.DataFrame(GetColumns(df_train, 'DOA'), columns=[df_train.columns[26]])), columns=[df_train.columns[26]])
DOA_COLUMN.head()

,DOA
0,0.011516
1,0.011516
2,0.011516
3,0.011516
4,0.011516


In [103]:
LastFew = pd.DataFrame(scaler.fit_transform(pd.DataFrame(GetColumns(df_train, 'LastFew'), columns=df_train.columns[27:32])), columns=df_train.columns[27:32])
LastFew['TWTT']=df_train['TWTT']
LastFew['AngleUncertainty']=df_train['AngleUncertainty']
LastFew['SampleRate']=df_train['SampleRate']
LastFew['Range']=df_train['Range']
LastFew.head()

,TWTT,Amplitude,AngleUncertainty,SampleRate,Range
0,0.000061,0.0,0,65108.40625,0.045832
1,0.000077,0.0,0,65108.40625,0.057291
2,0.000092,0.0,0,65108.40625,0.068749
3,0.000108,0.0,0,65108.40625,0.080207
4,0.000123,0.0,0,65108.40625,0.091665


In [104]:
df_train_scaled_properly = pd.concat([FirstFewScaled, IQandROLL, C_COLUMN, DOA_COLUMN, LastFew], axis=1)
df_train_scaled_properly.head()

,Ping#,Sample#,Port/Stbd,SampleTimeDelay,I1,Q1,I2,Q2,I3,Q3,I4,Q4,I5,Q5,I6,Q6,I7,Q7,I8,Q8,I9,Q9,I10,Q10,Roll,C,DOA,TWTT,Amplitude,AngleUncertainty,SampleRate,Range
0,7,5,0,0.000000,0.126465,0.039453,-0.081946,0.169626,-0.00804,0.072351,-0.072821,-0.014608,-0.02566,-0.024656,-0.049759,-0.005408,-0.052884,-0.020516,-0.067236,-0.119361,0.020121,-0.073706,0.132279,-0.075426,0.147727,0.90991,0.011516,0.000061,0.0,0,65108.40625,0.045832
1,7,6,0,0.000233,0.126465,0.039453,-0.081946,0.169626,-0.00804,0.072351,-0.072821,-0.014608,-0.02566,-0.024656,-0.049759,-0.005408,-0.052884,-0.020516,-0.067236,-0.119361,0.020121,-0.073706,0.132279,-0.075426,0.147727,0.90991,0.011516,0.000077,0.0,0,65108.40625,0.057291
2,7,7,0,0.000466,0.126465,0.039453,-0.081946,0.169626,-0.00804,0.072351,-0.072821,-0.014608,-0.02566,-0.024656,-0.049759,-0.005408,-0.052884,-0.020516,-0.067236,-0.119361,0.020121,-0.073706,0.132279,-0.075426,0.147727,0.90991,0.011516,0.000092,0.0,0,65108.40625,0.068749
3,7,8,0,0.000698,0.126465,0.039453,-0.081946,0.169626,-0.00804,0.072351,-0.072821,-0.014608,-0.02566,-0.024656,-0.049759,-0.005408,-0.052884,-0.020516,-0.067236,-0.119361,0.020121,-0.073706,0.132279,-0.075426,0.147727,0.90991,0.011516,0.000108,0.0,0,65108.40625,0.080207
4,7,9,0,0.000931,0.126465,0.039453,-0.081946,0.169626,-0.00804,0.072351,-0.072821,-0.014608,-0.02566,-0.024656,-0.049759,-0.005408,-0.052884,-0.020516,-0.067236,-0.119361,0.020121,-0.073706,0.132279,-0.075426,0.147727,0.90991,0.011516,0.000123,0.0,0,65108.40625,0.091665


In [105]:

#C_Array = GetColumns(df_train, 'C')

#C_Array.reshape(-1, 1)


FirstFewScaled = pd.DataFrame(scaler.fit_transform(GetColumns(df_train, 'FirstFew')), columns=df_train.columns[0:4])

df_train_properlyScaled = pd.concat([pd.DataFrame(scaler.fit_transform(pd.DataFrame(GetColumns(df_train, 'FirstFew'), columns=df_train.columns[0:4])), columns=df_train.columns[0:4]), pd.DataFrame(scaler_neg1_1.fit_transform(pd.DataFrame(GetColumns(df_train, 'IQandROLL'), columns=df_train.columns[4:24])), columns=df_train.columns[4:24]), pd.DataFrame(scaler.fit_transform(pd.DataFrame(GetColumns(df_train, 'C'), columns=[df_train.columns[25]])), columns=[df_train.columns[25]]), pd.DataFrame(scaler_neg1_1.fit_transform(pd.DataFrame(GetColumns(df_train, 'DOA'), columns=[df_train.columns[26]])), columns=[df_train.columns[26]]), pd.DataFrame(scaler.fit_transform(pd.DataFrame(GetColumns(df_train, 'LastFew'), columns=df_train.columns[27:32])), columns=df_train.columns[27:32])], axis=1)

df_train_properlyScaled['Ping#'] = df_train['Ping#']

df_train_properlyScaled['SampleRate'] = df_train['SampleRate']
df_train_properlyScaled['Sample#'] = df_train['Sample#']

#df_train_properlyScaled['Range'] = df_train['Range']

df_train_properlyScaled['Port/Stbd'] = df_train['Port/Stbd']

In [106]:
df_train_properlyScaled

,Ping#,Sample#,Port/Stbd,SampleTimeDelay,I1,Q1,I2,Q2,I3,Q3,I4,Q4,I5,Q5,I6,Q6,I7,Q7,I8,Q8,I9,Q9,I10,Q10,C,DOA,TWTT,Amplitude,AngleUncertainty,SampleRate,Range
0,7,5,0,0.000000,0.126465,0.039453,-0.081946,0.169626,-0.008040,0.072351,-0.072821,-0.014608,-0.025660,-0.024656,-0.049759,-0.005408,-0.052884,-0.020516,-0.067236,-0.119361,0.020121,-0.073706,0.132279,-0.075426,0.90991,0.011516,0.000000,0.000000,0.000000,65108.40625,6.302802e-08
1,7,6,0,0.000233,0.126465,0.039453,-0.081946,0.169626,-0.008040,0.072351,-0.072821,-0.014608,-0.025660,-0.024656,-0.049759,-0.005408,-0.052884,-0.020516,-0.067236,-0.119361,0.020121,-0.073706,0.132279,-0.075426,0.90991,0.011516,0.000233,0.000000,0.000000,65108.40625,2.328361e-04
2,7,7,0,0.000466,0.126465,0.039453,-0.081946,0.169626,-0.008040,0.072351,-0.072821,-0.014608,-0.025660,-0.024656,-0.049759,-0.005408,-0.052884,-0.020516,-0.067236,-0.119361,0.020121,-0.073706,0.132279,-0.075426,0.90991,0.011516,0.000466,0.000000,0.000000,65108.40625,4.656092e-04
3,7,8,0,0.000698,0.126465,0.039453,-0.081946,0.169626,-0.008040,0.072351,-0.072821,-0.014608,-0.025660,-0.024656,-0.049759,-0.005408,-0.052884,-0.020516,-0.067236,-0.119361,0.020121,-0.073706,0.132279,-0.075426,0.90991,0.011516,0.000698,0.000000,0.000000,65108.40625,6.983823e-04
4,7,9,0,0.000931,0.126465,0.039453,-0.081946,0.169626,-0.008040,0.072351,-0.072821,-0.014608,-0.025660,-0.024656,-0.049759,-0.005408,-0.052884,-0.020516,-0.067236,-0.119361,0.020121,-0.073706,0.132279,-0.075426,0.90991,0.011516,0.000931,0.000000,0.000000,65108.40625,9.311554e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5504452,1287,4297,0,0.999069,0.121114,0.044620,-0.083122,0.178632,-0.002254,0.079035,-0.063930,-0.020709,-0.027349,-0.039358,-0.061524,-0.009766,-0.058393,-0.013365,-0.058956,-0.116415,0.024596,-0.079996,0.130999,-0.075983,0.09009,-0.337935,0.999069,0.452381,0.807843,65108.40625,9.990012e-01
5504453,1287,4298,0,0.999302,0.113040,0.045212,-0.080127,0.186429,0.008223,0.079022,-0.060508,-0.026925,-0.034740,-0.042661,-0.069515,-0.005851,-0.058193,-0.003906,-0.050520,-0.111983,0.031353,-0.088027,0.122531,-0.087641,0.09009,-0.322072,0.999302,0.500000,0.380392,65108.40625,9.992340e-01
5504454,1287,4299,0,0.999534,0.118403,0.042763,-0.078292,0.179990,0.002376,0.074005,-0.069849,-0.025049,-0.038134,-0.032114,-0.062345,0.000133,-0.052623,-0.006683,-0.056262,-0.114607,0.029071,-0.085000,0.121772,-0.085909,0.09009,-0.317891,0.999534,0.571429,0.125490,65108.40625,9.994667e-01
5504455,1287,4300,0,0.999767,0.133273,0.036062,-0.078637,0.161131,-0.017923,0.066466,-0.081052,-0.008922,-0.020051,-0.017183,-0.040501,-0.010461,-0.054019,-0.027732,-0.075068,-0.117509,0.019956,-0.068154,0.132603,-0.071435,0.09009,-0.321873,0.999767,0.535714,0.188235,65108.40625,9.996995e-01


In [107]:
df_train_properlyScaled.describe()

,Ping#,Sample#,Port/Stbd,SampleTimeDelay,I1,Q1,I2,Q2,I3,Q3,I4,Q4,I5,Q5,I6,Q6,I7,Q7,I8,Q8,I9,Q9,I10,Q10,C,DOA,TWTT,Amplitude,AngleUncertainty,SampleRate,Range
count,5.504457e+06,5.504457e+06,5504457.0,5.504457e+06,5.504457e+06,5.504457e+06,5.504457e+06,5.504457e+06,5.504457e+06,5.504457e+06,5.504457e+06,5.504457e+06,5.504457e+06,5.504457e+06,5.504457e+06,5.504457e+06,5.504457e+06,5.504457e+06,5.504457e+06,5.504457e+06,5.504457e+06,5.504457e+06,5.504457e+06,5.504457e+06,5.504457e+06,5.504457e+06,5.504457e+06,5.504457e+06,5.504457e+06,5.504457e+06,5.504457e+06
mean,6.470000e+02,2.153000e+03,0.0,5.000000e-01,1.264778e-01,3.947868e-02,-8.194078e-02,1.696534e-01,-8.030354e-03,7.240320e-02,-7.282897e-02,-1.455575e-02,-2.568082e-02,-2.460683e-02,-4.977289e-02,-5.374336e-03,-5.291370e-02,-2.052685e-02,-6.724462e-02,-1.193843e-01,2.014490e-02,-7.372333e-02,1.322834e-01,-7.543048e-02,4.828450e-01,-1.786064e-01,5.000000e-01,5.813522e-01,3.578778e-01,6.510841e+04,4.999808e-01
std,3.697928e+02,1.240437e+03,0.0,2.887423e-01,4.173832e-02,4.735650e-02,5.484312e-02,4.701740e-02,5.302079e-02,4.711205e-02,4.642880e-02,5.031420e-02,5.382881e-02,5.293257e-02,4.906452e-02,5.209302e-02,5.393893e-02,5.669476e-02,5.302742e-02,4.998064e-02,5.144605e-02,5.363648e-02,5.004921e-02,5.371041e-02,1.926008e-01,2.220598e-01,2.887423e-01,1.587952e-01,3.035215e-01,0.000000e+00,2.887312e-01
min,7.000000e+00,5.000000e+00,0.0,0.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.510841e+04,0.000000e+00
25%,3.270000e+02,1.079000e+03,0.0,2.500000e-01,1.159875e-01,2.753108e-02,-9.599523e-02,1.576224e-01,-2.174575e-02,6.013396e-02,-8.487026e-02,-2.766957e-02,-3.969877e-02,-3.841471e-02,-6.237296e-02,-1.876422e-02,-6.648046e-02,-3.481784e-02,-8.084138e-02,-1.321630e-01,6.366749e-03,-8.807429e-02,1.188460e-01,-8.982469e-02,3.603604e-01,-2.977135e-01,2.500000e-01,5.119048e-01,1.372549e-01,6.510841e+04,2.499881e-01
50%,6.470000e+02,2.153000e+03,0.0,5.000000e-01,1.264651e-01,3.945315e-02,-8.194562e-02,1.696262e-01,-8.040446e-03,7.235058e-02,-7.282143e-02,-1.460816e-02,-2.566010e-02,-2.465603e-02,-4.975914e-02,-5.407642e-03,-5.288358e-02,-2.051621e-02,-6.723626e-02,-1.193608e-01,2.012133e-02,-7.370634e-02,1.322793e-01,-7.542623e-02,4.504505e-01,-2.243056e-01,5.000000e-01,6.071429e-01,2.392157e-01,6.510841e+04,4.999796e-01
75%,9.670000e+02,3.227000e+03,0.0,7.500000e-01,1.369545e-01,5.137521e-02,-6.789601e-02,1.816569e-01,5.710544e-03,8.459432e-02,-6.075939e-02,-1.489459e-03,-1.162142e-02,-1.085170e-02,-3.714532e-02,7.978488e-03,-3.930205e-02,-6.214589e-03,-5.367756e-02,-1.065877e-01,3.386089e-02,-5.938544e-02,1.456563e-01,-6.107296e-02,6.396396e-01,-8.399429e-02,7.500000e-01,6.904762e-01,4.745098e-01,6.510841e+04,7.499799e-01
max,1.287000e+03,4.301000e+03,0.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,6.510841e+04,1.000000e+00


In [109]:
#GoodSamplesPerPing = max(df_train['Sample#'])
GoodSamplesPerPing = GetPingNumber(7, 'train').shape[0]
print('GoodSamplesPerPing : %d' % GoodSamplesPerPing)
GoodSamplesPerPing_Test = GetPingNumber(7, 'test').shape[0]
print('GoodSamplesPerPing_Test : %d' % GoodSamplesPerPing_Test)
NumColumnsInputData = 1+20+1+1+1+1
#                   SampleTimeDelay + StaveData + DOA + Amplitude + C + Roll

NumColumnsOutputData = 1
#                   DOA


NumGoodPings = max(df_train['Ping#'])


#inputs = tf.keras.Input(shape=(GoodSamplesPerPing, NumColumnsInputData))
#lstm = tf.keras.layers.LSTM(units=GoodSamplesPerPing)
#correctOutputs = 
#outputs = lstm(inputs)
#print(outputs.shape)
#outputs = tf.keras.Output(shape=(GoodSamplesPerPing, NumColumnsOutputData))

GoodSamplesPerPing : 4297
GoodSamplesPerPing_Test : 4284


In [ ]:
#inputs.shape

#tf.keras.Input()

## **TimeseriesGenerator Test** 

In [110]:
#inputsArray = GetPingNumber(7, 'train').to_numpy()
#outputsArray = 
#batch_sizeSetting = GoodSamplesPerPing
# lengthSetting = 3
#generator = TimeseriesGenerator(inputsArray, outputsArray, lengthSetting, batch_sizeSetting)
#generator = TimeseriesGenerator()
#tf.keras.preprocessing.sequence.TimeseriesGenerator(inputs, outputs, lengthSetting, batch_sizeSetting)

#CurPingTrainData = df_train_scaled
CurPingTrainData = df_train_scaled_properly

#CurPingTrainData = GetPingNumber(7, 'train')

CurPingInputs = GetColumns(CurPingTrainData, 'InputData')
CurPingOutputs = GetColumns(CurPingTrainData, 'TrueData')

inputsArray = CurPingInputs.to_numpy()
outputsArray = CurPingOutputs.to_numpy()
outputsArray = outputsArray.reshape((len(outputsArray),1))
outputsArray = np.insert(outputsArray, 0, 0)    #shift target sample by one step.
outputsArray = np.delete(outputsArray, -1)


TestPingInputs = GetColumns(df_test_scaled, 'InputData')
TestPingOutputs = GetColumns(df_test_scaled, 'TrueData')
testInputsArray = TestPingInputs.to_numpy()

testOutputsArray = TestPingOutputs.to_numpy()
outputsArray = outputsArray.reshape((len(outputsArray),1))
testOutputsArray = np.insert(outputsArray, 0, 0)
testOutputsArray = np.delete(outputsArray, -1)


In [111]:
#######
print(testInputsArray.shape)
testInputsArray = testInputsArray.reshape((len(testInputsArray), 1, 22))
print(testInputsArray.shape)

(5487234, 22)
(5487234, 1, 22)


In [112]:
CurPingInputs.shape


(5504457, 22)

In [113]:
CurPingOutputs.shape

(5504457,)

In [114]:
inputsArray.shape

(5504457, 22)

In [115]:
outputsArray.shape

(5504457, 1)

In [ ]:
#inputsArray = np.append(inputsArray, np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]).reshape((1, 22)),axis=0)

In [ ]:
#inputsArray[5504456]
#inputsArray[5504457]

In [116]:
n_input = 2
#batch_sizeSetting = GoodSamplesPerPing
n_features = inputsArray.shape[1]
print('NumFeatures: %d' % n_features)

generator = TimeseriesGenerator(inputsArray, outputsArray, length=n_input, batch_size = GoodSamplesPerPing)

#number of Samples
print('Samples: %d' % len(generator))

x, y = generator[1]
np.set_printoptions(threshold=np.inf) # this is just so that when printing the arrays, all values are shown
print('%s => %s' % (x,y))

NumFeatures: 22
Samples: 1281
[[[ 0.00000000e+00  1.26465087e-01  3.94531460e-02 -8.19456205e-02
    1.69626175e-01 -8.04044588e-03  7.23505803e-02 -7.28214342e-02
   -1.46081577e-02 -2.56600967e-02 -2.46560331e-02 -4.97591401e-02
   -5.40764162e-03 -5.28835825e-02 -2.05162144e-02 -6.72362556e-02
   -1.19360765e-01  2.01213286e-02 -7.37063354e-02  1.32279300e-01
   -7.54262305e-02  0.00000000e+00]
  [ 2.32774674e-04  1.26465087e-01  3.94531460e-02 -8.19456205e-02
    1.69626175e-01 -8.04044588e-03  7.23505803e-02 -7.28214342e-02
   -1.46081577e-02 -2.56600967e-02 -2.46560331e-02 -4.97591401e-02
   -5.40764162e-03 -5.28835825e-02 -2.05162144e-02 -6.72362556e-02
   -1.19360765e-01  2.01213286e-02 -7.37063354e-02  1.32279300e-01
   -7.54262305e-02  0.00000000e+00]]

 [[ 2.32774674e-04  1.26465087e-01  3.94531460e-02 -8.19456205e-02
    1.69626175e-01 -8.04044588e-03  7.23505803e-02 -7.28214342e-02
   -1.46081577e-02 -2.56600967e-02 -2.46560331e-02 -4.97591401e-02
   -5.40764162e-03 -5.288

# [[data data data data ...... data data ]] => [data]

In [ ]:
x.shape

In [ ]:
y.shape

In [117]:
from keras.models import Sequential
from keras.layers import Dense 

#model = Sequential()
# define model

#model.add(LSTM(100, activation='tanh', input_shape=(n_input, n_features)))
#model.add(Dense(3))
#model.compile(optimizer='adam', loss='mse')


inputsLayer = tf.keras.Input(shape=(1, n_features))

LSTM_Layer = LSTM(100, activation='tanh', input_shape=(n_input, n_features))

intermediary1 = LSTM_Layer(inputsLayer)



outputsLayer = Dense(1)(intermediary1)
model = tf.keras.Model(inputs=inputsLayer, outputs=outputsLayer, name="MLDOA_LSTM_FUNCTIONAL_MODEL")
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model.summary()




Model: "MLDOA_LSTM_FUNCTIONAL_MODEL"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1, 22)]           0         
                                                                 
 lstm_4 (LSTM)               (None, 100)               49200     
                                                                 
 dense_4 (Dense)             (None, 1)                 101       
                                                                 
Total params: 49,301
Trainable params: 49,301
Non-trainable params: 0
_________________________________________________________________


In [118]:
totalSamples = len(df_train)
batchSize = GoodSamplesPerPing
stepsPerEpoch = totalSamples/batchSize
print(stepsPerEpoch)

1281.0


In [119]:
# fit model


model.fit_generator(generator, steps_per_epoch=stepsPerEpoch, epochs=125,
 verbose=1)



C:\Users\mbisbano\AppData\Local\Temp/ipykernel_35744/178129301.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator, steps_per_epoch=stepsPerEpoch, epochs=125,


Epoch 1/125
1281/1281 [==============================] - 12s 9ms/step - loss: 0.0273 - accuracy: 0.0000e+00
Epoch 2/125
1281/1281 [==============================] - 11s 8ms/step - loss: 0.0163 - accuracy: 0.0000e+00
Epoch 3/125
1281/1281 [==============================] - 11s 8ms/step - loss: 0.0120 - accuracy: 0.0000e+00
Epoch 4/125
1281/1281 [==============================] - 10s 8ms/step - loss: 0.0086 - accuracy: 0.0000e+00
Epoch 5/125
1281/1281 [==============================] - 11s 8ms/step - loss: 0.0076 - accuracy: 0.0000e+00
Epoch 6/125
1281/1281 [==============================] - 11s 8ms/step - loss: 0.0073 - accuracy: 0.0000e+00
Epoch 7/125
1281/1281 [==============================] - 10s 8ms/step - loss: 0.0071 - accuracy: 0.0000e+00
Epoch 8/125
1281/1281 [==============================] - 11s 8ms/step - loss: 0.0070 - accuracy: 1.8167e-07
Epoch 9/125
1281/1281 [==============================] - 11s 8ms/step - loss: 0.0069 - accuracy: 1.8167e-07
Epoch 10/125
1281/1281 [====

In [ ]:
model.layers

In [ ]:
predictOutputArray = model.predict(testInputsArray, verbose=1)

In [ ]:
predictOutputArray.shape

In [ ]:
plt.cla
plt.figure(1)
plt.plot(predictOutputArray[1:43000])
plt.title('Predicted Outputs')
plt.figure(2)
plt.plot(testOutputsArray[1:43000])
plt.title('True Outputs')

In [ ]:
inputsArray = inputsArray.reshape(len(inputsArray), 1, 22)
overfittingTestOutput = model.predict(inputsArray, verbose=1)

In [ ]:
plt.figure(3)
plt.plot(overfittingTestOutput[1:40000])
plt.title('Overfitting Test Predicted Outputs')
plt.figure(4)
plt.plot(outputsArray[1:40000])
plt.title('Overfitting Test True Outputs')

In [ ]:
x = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
y = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
y2 = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
y3 = np.array([2, 3, 4, 5, 6, 7, 8, 9, 10, 11])
y4 = np.array([3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
plt.figure(5)
plt.plot(x, y)
plt.plot(x, y2)
plt.plot(x, y3)
plt.plot(x, y4)